In [1]:
from pyNA.pyna import pyna
import openmdao.api as om
import numpy as np

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/laurensvoet/miniconda3/lib/python3.10/site-packages/pyoptsparse/pyOpt_MPI.py:68: UserWarning: mpi4py could not be imported. mpi4py is required to use the parallel gradient analysis and parallel objective analysis for non-gradient based optimizers. Continuing using a dummy MPI module from pyOptSparse.
  warnings.warn(warn)
/var/folders/6w/2tdqp9m179gg5blnzww9hgjc0000gn/T/ipykernel_30948/443738326.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Get NASA STCA trajectory from file
py_stca = pyna()
py_stca.trajectory.connect()
py_stca.noise.connect()
py_stca.calculate_time_history()
py_stca.plot_trajectory()

TypeError: 'noise' <class NoiseModel>: Error calling compute(), '<class 'jaxlib.xla_extension.DeviceArray'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html

In [3]:
# Compute NASA STCA trajectory using take-off model
z_cb = py_stca.problem.get_val('z')[np.where(py_stca.problem.get_val('tau') < 0.65)[0][0]]
v_max = py_stca.problem.get_val('v')[-1]
x_max = py_stca.problem.get_val('x')[-1]

# Set up pyNA
py = pyna(case_name='stca',
          engine_name='engine_derivative',
          trajectory_mode='model',
          z_cb=z_cb,
          v_max=v_max,
          x_max=x_max,
          pkrot=True)

# Initialize trajectory model
from pyNA.src.trajectory_model.sst.sst_take_off_trajectory import SSTTakeOffTrajectory

# Trajectory controls
controls = dict()
controls['k_rot'] = 1.2
controls['tau'] = {'groundroll':0.88, 'rotation':0.88, 'liftoff':0.88, 'vnrs':0.88, 'cutback':0.61}
controls['theta_flaps'] = {'groundroll':10., 'rotation':10., 'liftoff':10., 'vnrs':10., 'cutback':10.}
controls['theta_slats'] = {'groundroll':-6., 'rotation':-6., 'liftoff':-6., 'vnrs':-6., 'cutback':-6.}

model = SSTTakeOffTrajectory(settings=py.settings, aircraft=py.aircraft, controls=controls, mode='cutback', objective='t_end')
py.trajectory.connect(model=model)
py.noise.connect()
py.calculate_trajectory(objective='time')
py.plot_trajectory()



--- Constraint Report [phases] ---
    --- groundroll ---
        [final]   0.0000e+00 == flight_dynamics.v_rot_residual [m/s]
    --- rotation ---
        [final]   1.1000e+00 == flight_dynamics.n [None]
    --- liftoff ---
        [path]    0.0000e+00 <= flight_dynamics.gamma_dot  [deg/s]
        [path]    0.0000e+00 <= flight_dynamics.v_dot  [m/s**2]
    --- vnrs ---
        [path]    0.0000e+00 <= flight_dynamics.v_dot  [m/s**2]
        [path]    0.0000e+00 <= gamma  [deg]
    --- cutback ---
        [path]    0.0000e+00 <= flight_dynamics.v_dot  [m/s**2]



/Users/laurensvoet/miniconda3/lib/python3.10/site-packages/openmdao/recorders/sqlite_recorder.py:227: UserWarning:The existing case recorder file, dymos_solution.db, is being overwritten.


Model viewer data has already been recorded for Driver.
Model viewer data has already been recorded for Driver.


TypeError: 'noise' <class NoiseModel>: Error calling compute(), '<class 'jaxlib.xla_extension.DeviceArray'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html